In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler
from models.resnet20 import build_resnet20  # Make sure this path is correct

import os

# Create model directory if not exists
os.makedirs("models", exist_ok=True)

# -------------------------
# One-Cycle Learning Rate Scheduler
# -------------------------
def one_cycle_lr_schedule(initial_lr, max_lr, cycle_length, step_size):
    def lr_schedule(epoch):
        cycle_epoch = epoch % cycle_length
        if cycle_epoch < step_size:
            return initial_lr + (max_lr - initial_lr) * (cycle_epoch / step_size)
        else:
            return max_lr - (max_lr - initial_lr) * ((cycle_epoch - step_size) / step_size)
    return lr_schedule

# -------------------------
# One-Cycle Pruning (before training)
# -------------------------
def one_cycle_prune(model, sparsity):
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D):
            weights = layer.get_weights()
            if len(weights) < 2:
                continue  # skip if bias or weight is missing
            kernel, bias = weights
            abs_weight = np.abs(kernel)
            threshold = np.percentile(abs_weight, sparsity * 100)
            pruned_kernel = np.where(abs_weight < threshold, 0, kernel)
            layer.set_weights([pruned_kernel, bias])
    print(f"One-cycle pruning applied at sparsity {sparsity * 100:.1f}%.")

# -------------------------
# Train the Model
# -------------------------
def train_model(sparsity=0.5, batch_size=128, epochs=150):
    model_path = f"models/pruned_resnet20_onecycle_sparsity_{sparsity}.h5"
    if os.path.exists(model_path):
        print(f"Model already exists at {model_path}. Skipping training.")
        return None

    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

    model = build_resnet20()
    model.build(input_shape=(None, 32, 32, 3))

    one_cycle_prune(model, sparsity)

    step_size = epochs // 2
    lr_schedule = one_cycle_lr_schedule(initial_lr=1e-6, max_lr=1e-2, cycle_length=epochs, step_size=step_size)
    lr_scheduler = LearningRateScheduler(lr_schedule)

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(x_test, y_test),
                        callbacks=[lr_scheduler])

    model.save(model_path)
    print(f"Model saved to {model_path}")
    return history

In [7]:
history = train_model(sparsity=0.3, batch_size=128, epochs=150)

One-cycle pruning applied at sparsity 30.0%.
Epoch 1/150
391/391 [==============================] - 34s 79ms/step - loss: 3.8804 - accuracy: 0.1092 - val_loss: 3.4101 - val_accuracy: 0.1154 - lr: 1.0000e-06
Epoch 2/150
391/391 [==============================] - 30s 75ms/step - loss: 1.9473 - accuracy: 0.3611 - val_loss: 1.6759 - val_accuracy: 0.4373 - lr: 1.3432e-04
Epoch 3/150
391/391 [==============================] - 29s 75ms/step - loss: 1.5194 - accuracy: 0.5007 - val_loss: 1.5205 - val_accuracy: 0.5104 - lr: 2.6764e-04
Epoch 4/150
391/391 [==============================] - 29s 75ms/step - loss: 1.3183 - accuracy: 0.5781 - val_loss: 1.4450 - val_accuracy: 0.5308 - lr: 4.0096e-04
Epoch 5/150
391/391 [==============================] - 29s 75ms/step - loss: 1.1734 - accuracy: 0.6338 - val_loss: 1.2116 - val_accuracy: 0.6105 - lr: 5.3428e-04
Epoch 6/150
391/391 [==============================] - 29s 75ms/step - loss: 1.0653 - accuracy: 0.6729 - val_loss: 1.4428 - val_accuracy: 0.5664 

In [2]:
%%timeit
history = train_model(sparsity=0.5, batch_size=128, epochs=150)

One-cycle pruning applied at sparsity 50.0%.
Epoch 1/150
391/391 [==============================] - 36s 76ms/step - loss: 3.7287 - accuracy: 0.1319 - val_loss: 3.4760 - val_accuracy: 0.1262 - lr: 1.0000e-06
Epoch 2/150
391/391 [==============================] - 29s 74ms/step - loss: 1.9754 - accuracy: 0.3496 - val_loss: 1.7273 - val_accuracy: 0.4160 - lr: 1.3432e-04
Epoch 3/150
391/391 [==============================] - 29s 74ms/step - loss: 1.5462 - accuracy: 0.4874 - val_loss: 1.7558 - val_accuracy: 0.4179 - lr: 2.6764e-04
Epoch 4/150
391/391 [==============================] - 29s 74ms/step - loss: 1.3613 - accuracy: 0.5598 - val_loss: 1.5984 - val_accuracy: 0.4858 - lr: 4.0096e-04
Epoch 5/150
391/391 [==============================] - 29s 74ms/step - loss: 1.2104 - accuracy: 0.6150 - val_loss: 1.5041 - val_accuracy: 0.5257 - lr: 5.3428e-04
Epoch 6/150
391/391 [==============================] - 29s 74ms/step - loss: 1.0838 - accuracy: 0.6643 - val_loss: 1.2411 - val_accuracy: 0.6099 

In [3]:
%%timeit
history = train_model(sparsity=0.7, batch_size=128, epochs=150)

One-cycle pruning applied at sparsity 70.0%.
Epoch 1/150
391/391 [==============================] - 31s 75ms/step - loss: 3.1843 - accuracy: 0.1182 - val_loss: 2.8370 - val_accuracy: 0.1330 - lr: 1.0000e-06
Epoch 2/150
391/391 [==============================] - 29s 73ms/step - loss: 1.8723 - accuracy: 0.3650 - val_loss: 1.7795 - val_accuracy: 0.3816 - lr: 1.3432e-04
Epoch 3/150
391/391 [==============================] - 29s 73ms/step - loss: 1.4750 - accuracy: 0.5069 - val_loss: 1.4851 - val_accuracy: 0.5068 - lr: 2.6764e-04
Epoch 4/150
391/391 [==============================] - 28s 72ms/step - loss: 1.2738 - accuracy: 0.5834 - val_loss: 1.5786 - val_accuracy: 0.4974 - lr: 4.0096e-04
Epoch 5/150
391/391 [==============================] - 29s 73ms/step - loss: 1.1221 - accuracy: 0.6417 - val_loss: 1.5522 - val_accuracy: 0.5344 - lr: 5.3428e-04
Epoch 6/150
391/391 [==============================] - 28s 73ms/step - loss: 1.0097 - accuracy: 0.6814 - val_loss: 1.5449 - val_accuracy: 0.4940 

In [4]:
%%timeit
history = train_model(sparsity=0.9, batch_size=128, epochs=150)

One-cycle pruning applied at sparsity 90.0%.
Epoch 1/150
391/391 [==============================] - 31s 74ms/step - loss: 3.6229 - accuracy: 0.1030 - val_loss: 2.8379 - val_accuracy: 0.1009 - lr: 1.0000e-06
Epoch 2/150
391/391 [==============================] - 29s 73ms/step - loss: 1.8333 - accuracy: 0.3735 - val_loss: 1.8659 - val_accuracy: 0.3346 - lr: 1.3432e-04
Epoch 3/150
391/391 [==============================] - 29s 73ms/step - loss: 1.3754 - accuracy: 0.5216 - val_loss: 1.4491 - val_accuracy: 0.5025 - lr: 2.6764e-04
Epoch 4/150
391/391 [==============================] - 28s 73ms/step - loss: 1.1439 - accuracy: 0.6100 - val_loss: 1.2070 - val_accuracy: 0.5896 - lr: 4.0096e-04
Epoch 5/150
391/391 [==============================] - 29s 73ms/step - loss: 0.9790 - accuracy: 0.6721 - val_loss: 1.1218 - val_accuracy: 0.6276 - lr: 5.3428e-04
Epoch 6/150
391/391 [==============================] - 29s 73ms/step - loss: 0.8585 - accuracy: 0.7178 - val_loss: 1.0611 - val_accuracy: 0.6530 